In [ ]:
import seaborn as sns
sns.set()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, cross_val_score 
import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 1. Read the Data

In [ ]:
train_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

## 2. Data exploration

#### Check NaN

In [ ]:
#Copy the train_data
train_Data = train_data.copy() 

#Check NaN
train_Data.isna().sum() 

## 3. Create a pivot tabel

In [ ]:
train_Data = train_Data.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum') 

In [ ]:
train_Data.head() 

In [ ]:
train_Data.columns

In [ ]:
#Combine the test_data to train_Data_Nov for prediction
test_Data = test_data.copy()
test_Data = test_Data.pivot_table(index = ['shop_id','item_id'],fill_value = 0)
Combine_train_test = pd.merge(test_Data, train_Data, how = 'left', on = ['shop_id','item_id']).fillna(0)

In [ ]:
Combine_train_test = Combine_train_test.sort_values(by = 'ID')

In [ ]:
Combine_train_test.head(10)

## 4. Deep Learning

### Split the training_data and training_target

In [ ]:
train_data.shape

In [ ]:
#Drop the ID
Combine_train_test = Combine_train_test.drop(columns = ['ID'])

#Train Data
train_data = np.array(Combine_train_test.values[:,:-1]).reshape(np.array(Combine_train_test.values[:,:-1]).shape[0],np.array(Combine_train_test.values[:,:-1]).shape[1], 1)
#The target 
train_target = Combine_train_test.values[:,-1:]

test_data = np.array(Combine_train_test.values[:,1:]).reshape(np.array(Combine_train_test.values[:,1:]).shape[0],np.array(Combine_train_test.values[:,1:]).shape[1], 1)

### Bulid the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, TimeDistributed, Flatten

In [ ]:
#LSTM
model = Sequential()
model.add(LSTM(55, return_sequences = True, input_shape = (train_data.shape[1], 1)))
model.add(LSTM(55, return_sequences = True, ))
model.add(LSTM(55))
model.add(Dense(25))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse'])
#Conv1D
#def model_Conv1D():
    #model = Sequential()
    #model.add(Conv1D(filters = 32, kernel_size = 2, activation = 'relu', input_shape = (train_data.shape[1],train_data.shape[2], )))
    #model.add(MaxPooling1D(pool_size = 2))
    #model.add(Conv1D(filters = 50, kernel_size = 2, activation = 'relu'))
    #model.add(MaxPooling1D(pool_size = 2))
    #model.add(Flatten())
    #model.add(Dense(50))
    #model.add(Dropout(rate = 0.3))
    #model.add(Dense(1))
    #model.compile(optimizer= 'rmsprop', loss = 'mse', metrics = ['mse'])
    #return model

#Conv1D_LSTM
#def model_Conv1D_LSTM():
    #subsequences := 3
    #model = Sequential()
    #model.add(TimeDistributed(Conv1D(filters = 32, kernel_size = 2, activation = 'relu', input_shape = (None, train_data.shape[1] // 3,train_data.shape[2], ))))
    #model.add(TimeDistributed(MaxPooling1D(pool_size = 2)))
    #model.add(TimeDistributed(Flatten()))
    #model.add(LSTM(30))
    #model.add(Dropout(rate = 0.3))
    #model.add(Dense(1))
    #model.compile(optimizer= 'rmsprop', loss = 'mse', metrics = ['mse'])
    #return model
    
    
    

In [ ]:
#Model 
history_1 = model.fit(train_data, train_target, epochs = 10, batch_size = 4)
df_his1 = pd.DataFrame(history_1.history)

#Model 2
#history_2 = model_Conv1D().fit(train_data, train_target, epochs = 15)
#df_his2 = pd.DataFrame(history_2.history)

#Model 3
#train_data_1 = train_data.reshape((train_data.shape[0], 3 , train_data.shape[1] // 3 , 1))
#history_3 = model_Conv1D_LSTM().fit(train_data_1, train_target, epochs = 15)
#df_his3 = pd.DataFrame(history_3.history)



### Mean Square Roots and Epochs

In [ ]:
plt.figure(figsize = (10 , 10))
sns.lineplot(x= df_his1.index + 1, y = df_his1['loss'], color='r', label="LSTM loss")
#sns.lineplot(x= df_his2.index + 1, y= df_his2['loss'], color='g', label="Conv1D loss")
#sns.lineplot(x= df_his3.index + 1, y= df_his3['loss'], color='b', label="Conv1D+LSTM loss")
#plt.legend()
#plt.show()

In [ ]:
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
submission['item_cnt_month'] = model.predict(test_data).clip(0,30).round(0)
submission.to_csv('submission.csv', index=False)

In [ ]:
submission